In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
 
from ipywidgets import interact
from scipy.integrate import odeint, solve_ivp
from matplotlib import cm

<hr style="border:2px solid black"> </hr>

Consider a flow past a cylinder of radius 1 with the free stream from the left:

$$ \Phi = \Phi_{F.S.} + \Phi_{Doublet} = \alpha\bigg[z + \frac{1}{z}\bigg]$$



$$ u = \alpha \bigg[1 - \frac{(x^2 - y^2)}{(x^2 + y^2)^2}\bigg], \ \ v = \frac{-2\alpha xy}{(x^2 + y^2)^2} $$



$$ \therefore |V| = \alpha\sqrt{1 + \frac{1 - 2(x^2 - y^2)}{(x^2 + y^2)^2}} = \alpha\sqrt{1 + \frac{1 - 2r^2\cos{2\theta}}{r^4}}$$

Coefficient of Pressure:


$$ C_p = \frac{P - P_\infty}{\frac{1}{2}\rho v_\infty^2} $$

Since $ P_\infty + \frac{1}{2}\rho v_\infty^2 \ = \ P + \frac{1}{2}\rho v^2 $ (Bernoulli's Equation):

$$ C_p = \frac{P - P_\infty}{\frac{1}{2}\rho v_\infty^2} = \frac{v_\infty^2 - v^2}{v_\infty^2} = 1 - \frac{v^2}{v_\infty^2} $$

In [2]:
# Cp and V plots at the surface of the cylinder of radius 1:
def cp_v_plot(alpha, Plot):
    fig, ax = plt.subplots(nrows = 1, ncols = 1)
    
    theta = np.linspace(0, 2*np.pi, 1000)
    r = 1
    
    v = alpha*np.sqrt(1 + ((1 - 2*r*r*np.cos(2*theta))/(r**4)))
    cp = 1 - (v/alpha)**2
    
    if Plot == 'Velocity vs Theta':
        ax.plot(theta, v)
        ax.set_title(Plot)
        ax.set_xlabel('$\Theta$')
        ax.set_ylabel('V')
        ax.axis('equal')
    
    elif Plot == 'CP vs Theta':
        ax.set_ylabel('$C_p$')
        ax.set_xlabel('$\Theta$')
        ax.set_label(Plot)
        ax.plot(theta, cp)
        ax.axis('equal')

interact(cp_v_plot, alpha = (0, 2, 0.5), Plot = ['Velocity vs Theta', 'CP vs Theta']);

interactive(children=(FloatSlider(value=1.0, description='alpha', max=2.0, step=0.5), Dropdown(description='Pl…

<hr style="border:2px solid black"> </hr>

Adding a vortex of strength $\Gamma$ to the flow model $(\beta = \frac{\Gamma}{2\pi}) $:

$$ \Phi = \Phi_{F.S.} + \Phi_{Doublet} + \Phi_{Vortex} = \alpha\bigg[z + \frac{1}{z}\bigg] - i\beta\ln{z}$$



$$ u = \alpha \bigg[1 - \frac{(x^2 - y^2)}{(x^2 + y^2)^2}\bigg] + \frac{-\beta y}{(x^2 + y^2)}, \ \ v = \frac{-2\alpha xy}{(x^2 + y^2)^2} + \frac{\beta x}{(x^2 + y^2)} $$


In [3]:
# Cp, V at the cylinder surface, Streamlines and Potential Lines outside the cylinder, Pressure field:
# Rho, P_inf assumed to be 1
def vortex_cylinder_plots(alpha, Gamma, Plot):
    
    beta = Gamma/(2*np.pi)
    x, y = np.mgrid[-3:3:50j, -3:3:50j]
    z = x + 1j*y
    
    Phi = alpha*(z + (1/z)) - 1j*beta*np.log(z)
    vz = alpha*(1 - 1/(z**2)) - 1j*(beta/z)
    u, v = vz.real, -vz.imag
    
    if Plot == 'Velocity vs Theta':
        theta = np.linspace(0, 2*np.pi, 100)
        v = np.sqrt((2*alpha*np.sin(theta) - beta)**2)
        plt.figure(figsize = (10, 10))
        plt.plot(theta, v)
        pass
    
    elif Plot == 'Streamlines':
        plt.figure(figsize = (10, 10))
        plt.contour(x, y, Phi.imag, 50)
        plt.contour(x, y, Phi.imag, [0], colors = 'Red')
        
    elif Plot == 'Potential Lines':
        plt.figure(figsize = (10, 10))
        plt.contour(x, y, Phi.real, 50)
        
    elif Plot == 'Pressure Field':
        x, y = np.mgrid[-2.5:2.5:100j, -2.5:2.5:100j]
        beta = Gamma/(2*np.pi)
        
        z = x + 1j*y
    
        Phi = alpha*(z + (1/z)) - 1j*beta*np.log(z)
        vz = alpha*(1 - 1/(z**2)) - 1j*(beta/z)
        u, v = vz.real, -vz.imag
        
        
        p = 1 + 0.5*(np.absolute(vz))**2
        p[np.sqrt(x**2 + y**2) < 0.5] = 0
        plt.figure(figsize = (10, 10))
        plt.contour(x, y, p, 10)
    
    elif Plot == 'CP vs Theta':
        theta = np.linspace(0, 2*np.pi, 100)
        cp = 1 - (2*np.sin(theta) + (beta/alpha))**2
        plt.figure(figsize = (10, 10))
        plt.plot(theta, cp)
        
interact(vortex_cylinder_plots, alpha = (0, 2, 0.5), Gamma = (0, 2, 0.5), Plot = ['Streamlines', 'Potential Lines', 'Pressure Field', 'Velocity vs Theta', 'CP vs Theta']);

interactive(children=(FloatSlider(value=1.0, description='alpha', max=2.0, step=0.5), FloatSlider(value=1.0, d…

Force on the cylinder:

$$F =  \oint_{C} \frac{1}{2}\rho [v_\infty^2 - v^2 ] d\vec{A}  = \int_o^{2\pi} \frac{1}{2}[v_\infty^2 - v^2](-cos\theta \hat{i} - sin\theta \hat{j})d\theta$$

$$ v(r, \theta)  =  \bigg[\alpha\bigg[1 - \frac{\cos{2\theta}}{r^2}\bigg] - \frac{\beta\sin{\theta}}{r}\bigg] \hat{i} + \bigg[-\frac{\alpha\sin{2\theta}}{r^2} + \frac{\beta\cos{\theta}}{r}\bigg]\hat{j}$$



In [4]:
def integral(alpha, Gamma):
    beta = Gamma/(2*np.pi)
    dl = 0.001
    theta = np.arange(0, 2*np.pi, dl)
    lift = 0
    drag = 0
    for i in range(len(theta)):
        if i < len(theta) - 1:
            theta_i = (theta[i] + theta[i + 1])/2
        else:
            theta_i = theta[i]
        c1 = np.cos(theta_i)
        c2 = np.cos(2*theta_i)
        s1 = np.sin(theta_i)
        s2 = np.sin(2*theta_i)
        lift += -s1*dl*(0.5*(alpha**2 - ((alpha - alpha*c2 - beta*s1)**2 + (alpha*s2 - beta*c1)**2)))
        drag += -c1*dl*(0.5*(alpha**2 - ((alpha - alpha*c2 - beta*s1)**2 + (alpha*s2 - beta*c1)**2)))
    print('The Lift force experienced by the cylinder at alpha =', alpha, 'Gamma =', Gamma, 'is equal to ', lift, '\n')
    print('The Drag force experienced by the cylinder at alpha =', alpha, 'Gamma =', Gamma, 'is equal to ', drag, '\n')
interact(integral, alpha = (0, 2, 0.5), Gamma = (0, 2, 0.5));

interactive(children=(FloatSlider(value=1.0, description='alpha', max=2.0, step=0.5), FloatSlider(value=1.0, d…